<a href="https://colab.research.google.com/github/oytungunes/ResearchBilkent/blob/main/PeopleWalking_Range_Doppler_and_Spectrogram_andRangeTime_BiLSTM_twoclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Burada sadece range-doppler 2D image datası ve spectrogram datası LSTM ile train edildi.
# range doppler , spectrogram ve rangetime bi-lstm encoder ile train edildi, augmentation yok

#Mean test accuracy is 0.857, mean test f1 score is 0.853, max test accuracy is 0.895, max test f1 score is 0.894, min test accuracy is 0.807, min test f1 score is 0.797, std of test accuracy is 0.037, std of test f1 score is 0.041
#Time elapsed through all process: 369.855, sec
#

In [6]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, GRU, TimeDistributed, Bidirectional
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
import gc
import os
from keras.layers import LeakyReLU

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:


# Load Range-Doppler fast data
range_doppler_fast = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_fast.mat')
range_doppler_fast = range_doppler_fast['range_doppler_fast']
range_doppler_fast = np.transpose(range_doppler_fast, (2, 0, 1)) # label x width x height
range_doppler_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_fast_label.mat')
range_doppler_fast_label = range_doppler_fast_label['range_doppler_fast_label']

# Load Range-Doppler slow data

range_doppler_slow = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_slow.mat')
range_doppler_slow = range_doppler_slow['range_doppler_slow']
range_doppler_slow = np.transpose(range_doppler_slow, (2, 0, 1))# label x width x height
range_doppler_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/range_doppler_slow_label.mat')
range_doppler_slow_label = range_doppler_slow_label['range_doppler_slow_label']

# Load Spectrogram fast data
spectrogram_fast_resized = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_resized.mat')
spectrogram_fast_resized = spectrogram_fast_resized['spectrogram_fast_resized']
spectrogram_fast_resized = np.transpose(spectrogram_fast_resized, (2, 0, 1)) # label x width x height
spectrogram_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_label.mat')
spectrogram_fast_label = spectrogram_fast_label['spectrogram_fast_label']


# Load Spectrogram slow data
spectrogram_slow_resized= scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_resized.mat')
spectrogram_slow_resized = spectrogram_slow_resized['spectrogram_slow_resized']
spectrogram_slow_resized = np.transpose(spectrogram_slow_resized, (2, 0, 1)) # label x width x height
spectrogram_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_label.mat')
spectrogram_slow_label = spectrogram_slow_label['spectrogram_slow_label']




# Load RangeTime fast data
rangetime_fast= scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/rangetime_fast.mat')
rangetime_fast = rangetime_fast['rangetime_fast']
rangetime_fast = np.transpose(rangetime_fast, (2, 0, 1)) # label x width x height
rangetime_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/rangetime_fast_label.mat')
rangetime_fast_label = rangetime_fast_label['rangetime_fast_label']

# Load RangeTime slow data
rangetime_slow = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/rangetime_slow.mat')
rangetime_slow = rangetime_slow['rangetime_slow']
rangetime_slow = np.transpose(rangetime_slow, (2, 0, 1)) # label x width x height
rangetime_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/rangetime_slow_label.mat')
rangetime_slow_label = rangetime_slow_label['rangetime_slow_label']





In [9]:
# Concat range-doppler data
range_doppler_concat = np.concatenate((range_doppler_fast,range_doppler_slow),axis=0) # fast =fall, slow=walking
range_doppler_concat = range_doppler_concat[:,:,:,np.newaxis]
range_doppler_concat_label = np.zeros((range_doppler_concat.shape[0],1))
range_doppler_concat_label[:range_doppler_fast.shape[0],:] = 1
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]


# Concat spectrogram data
spectrogram_concat = np.concatenate((spectrogram_fast_resized,spectrogram_slow_resized),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis]
spectrogram_concat_label = np.zeros((spectrogram_concat.shape[0],1))
spectrogram_concat_label[:spectrogram_fast_resized.shape[0],:] = 1
# Shuffle concat spectrogram
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]

# Concat rangetime data
rangetime_concat = np.concatenate((rangetime_fast,rangetime_slow),axis=0)
rangetime_concat = rangetime_concat[:,:,:,np.newaxis]
rangetime_concat_label = np.zeros((rangetime_concat.shape[0],1))
rangetime_concat_label[:rangetime_fast.shape[0],:] = 1

# Shuffle concat data
shuffle_indx = random.sample(range(0, rangetime_concat.shape[0]), rangetime_concat.shape[0]) # split validation data

rangetime_concat_shuffle = rangetime_concat[shuffle_indx,:,:,:]
rangetime_concat_label_shuffle = rangetime_concat_label[shuffle_indx,:]

In [10]:
def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,rangetime_concat_shuffle_train,label_shuffle_train):

  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):
    randomlist_for_validation_indx

  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  wavelet_validation_data = rangetime_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]

  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  rangetime_training_data = rangetime_concat_shuffle_train[randomlist_for_train_indx,:,:,:]


  #get labels
  validation_labels = label_shuffle_train[randomlist_for_validation_indx,:]
  training_labels = label_shuffle_train[randomlist_for_train_indx,:]


  # get CLASS1 and CLASS2 indexes of training dat

  class1_indexes = np.where(training_labels == 0)[0]
  class2_indexes = np.delete(range(0, training_labels.shape[0]), class1_indexes)


  class1_spectrograms_train = spectrogram_training_data[class1_indexes,:,:,:]
  class2_spectrograms_train = spectrogram_training_data[class2_indexes,:,:,:]


  class1_rangedoppler_train = range_doppler_training_data[class1_indexes,:,:,:]
  class2_rangedoppler_train = range_doppler_training_data[class2_indexes,:,:,:]

  class1_rangetime_train = rangetime_training_data[class1_indexes,:,:,:]
  class2_rangetime_train = rangetime_training_data[class2_indexes,:,:,:]

  size_of_samples_class1= class1_spectrograms_train.shape[0]
  size_of_samples_class2 = class2_spectrograms_train.shape[0]


  class2_label = np.ones((size_of_samples_class2,1))
  class1_label = np.zeros((size_of_samples_class1,1))


  # unify data
  spectrogram_training_data = np.concatenate((class2_spectrograms_train,class1_spectrograms_train),axis=0)
  range_doppler_training_data = np.concatenate((class2_rangedoppler_train,class1_rangedoppler_train),axis=0)
  wavelet_training_data = np.concatenate((class2_rangetime_train,class1_rangetime_train),axis=0)


  training_labels = np.concatenate((class2_label,class1_label),axis=0)


  return (spectrogram_training_data,range_doppler_training_data,wavelet_training_data,training_labels,\
     spectrogram_validation_data,range_doppler_validation_data,wavelet_validation_data, validation_labels)

In [11]:
def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
wavelet_concat_shuffle = normalize_inputs(rangetime_concat_shuffle, normalize_inputs_enable)

range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)


In [12]:
n_features = range_doppler_concat_shuffle.shape[1]
n_steps = range_doppler_concat_shuffle.shape[2]
range_doppler_concat_shuffle = np.transpose(range_doppler_concat_shuffle, axes = (0,2,1,3))
spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3))
rangetime_concat_shuffle = np.transpose(rangetime_concat_shuffle, axes = (0,2,1,3))


In [13]:
print(range_doppler_concat_shuffle.shape)
print(spectrogram_concat_shuffle.shape)
print(rangetime_concat_shuffle.shape)

(114, 100, 100, 1)
(114, 100, 100, 1)
(114, 100, 100, 1)


In [14]:
t = time.time()
# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 64
dropout_prob_dense = 0.5
repeat_of_mixup = 5
number_of_repeat = 3
unit_number_of_lstm = 16
dense_unit_of_range_doppler_function = 256
dense_unit_of_spectrogram_function = 8
dense_unit_of_wavelet=8
decoder_dense_unit = 256
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del range_doppler_concat_shuffle_test
    del spectrogram_concat_shuffle_test
    del range_doppler_training_data
    del range_doppler_concat_shuffle_train
    del spectrogram_concat_shuffle_train
    del spectrogram_training_data


  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):
    gc.collect()
    K.clear_session()

    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    rangetime_concat_shuffle_test = rangetime_concat_shuffle[randomlist_for_test_indx,:,:,:]

    range_doppler_concat_label_shuffle_test = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]


    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    rangetime_concat_shuffle_train =rangetime_concat_shuffle[randomlist_for_train_indx,:,:,:]
    label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]



      # ---------------- MixUp Augmentation ----------------
    (spectrogram_training_data,range_doppler_training_data,rangetime_training_data,training_labels,\
     spectrogram_validation_data,range_doppler_validation_data,rangetime_validation_data, validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,rangetime_concat_shuffle_train,label_shuffle_train)

    # ---------------- Neural Network Architecture ----------------



    def lstm_encoder_network_1(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_range_doppler_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def lstm_encoder_network_2(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_spectrogram_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)



    def lstm_encoder_network_3(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_wavelet)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(decoder_dense_unit)(input)
      x = BatchNormalization()(x)
      x = LeakyReLU()(x)
      x = Dropout(0.3)(x)
      x = Dense(dense_size)(x)
      x = LeakyReLU()(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(1, activation="sigmoid")(x)
      return Model(input, x)

    input_shape = range_doppler_concat_shuffle.shape[1:3]
    base_network_lstm = lstm_encoder_network_1(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_lstm(range_doppler_input)

    input_shape = spectrogram_concat_shuffle_train.shape[1:3]
    base_network_lstm_2 = lstm_encoder_network_2(input_shape)
    spectrogram_input  = Input(shape=input_shape)
    processed_spectrogram  = base_network_lstm_2(spectrogram_input)

    input_shape = rangetime_concat_shuffle_train.shape[1:3]
    base_network_lstm_3 = lstm_encoder_network_3(input_shape)
    rangetime_input  = Input(shape=input_shape)
    processed_wavelet = base_network_lstm_3(rangetime_input)

    concat_layer = Concatenate()([processed_range_doppler, processed_spectrogram, processed_wavelet])


    base_decoder_network = decoder_for_concat((concat_layer.shape[1],))
    out = base_decoder_network(concat_layer)


    model = Model(inputs=[range_doppler_input, spectrogram_input,rangetime_input], outputs=[out])


    if repeat_run_number == 0:
      print(base_network_lstm.summary())
      print(base_network_lstm_2.summary())
      print(base_decoder_network.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0,restore_best_weights=True, mode='min')

    history = model.fit((range_doppler_training_data, spectrogram_training_data,rangetime_training_data),(training_labels),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((range_doppler_validation_data, spectrogram_validation_data,rangetime_validation_data) , (validation_labels)))

    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test, spectrogram_concat_shuffle_test,rangetime_concat_shuffle_test],\
                                               [range_doppler_concat_label_shuffle_test],batch_size=batch_size)
    gc.collect()
    # ---------------- Get Test Results ----------------

    y_test_predicted = model.predict((range_doppler_concat_shuffle_test, spectrogram_concat_shuffle_test,rangetime_concat_shuffle_test), batch_size=batch_size)


    # ----- Binarize y_test_predicted values -----
    y_test_predicted_binary = np.zeros(y_test_predicted.size)
    for ii in range(y_test_predicted.size):
      if y_test_predicted[ii] < 0.5:
        y_test_predicted_binary[ii] = 0
      else:
        y_test_predicted_binary[ii] = 1

    test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(range_doppler_concat_label_shuffle_test, y_test_predicted_binary, average='macro')

    test_accuracy_per_fold.append(test_accuracy)
    f1_score_per_fold.append(test_f1_score)
    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  print(f1_score_per_run)
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.3f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.3f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.3f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.3f}".format(np.std(f1_score_per_run, axis=0))}')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 32)        │        14,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 3200)           │        12,800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 848,256 (3.24 MB)

 Trainable params: 841,344 (3.21 MB)

 Non-trainable params: 6,912 (27.00 KB)

...

 Trainable params: 86,913 (339.50 KB)

 Non-trainable params: 512 (2.00 KB)

None
Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 539ms/step - accuracy: 0.4922 - loss: 0.9354 - val_accuracy: 0.6667 - val_loss: 0.5712
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5515 - loss: 0.8617 - val_accuracy: 0.8333 - val_loss: 0.4878
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5864 - loss: 0.7903 - val_accuracy: 0.9444 - val_loss: 0.4366
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5921 - loss: 0.8720 - val_accuracy: 0.8889 - val_loss: 0.4160
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6105 - loss: 0.8660 - val_accuracy: 0.8333 - val_loss: 0.4282
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.6109 - loss: 0.7439 - val_accuracy: 0.7778 - val_loss: 0.4235
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5853 - loss: 0.8529 - val_accuracy: 0.7222 - val_loss: 0.4506
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6407 - loss: 0.7200 - val_accuracy: 0.7222 - val

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 788ms/step
[0.8071146130561828]
[0.7966213932266563]
Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 537ms/step - accuracy: 0.5161 - loss: 0.9104 - val_accuracy: 0.7778 - val_loss: 0.6231
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.5415 - loss: 0.8798 - val_accuracy: 0.8889 - val_loss: 0.5455
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.4673 - loss: 1.1132 - val_accuracy: 0.8889 - val_loss: 0.4873
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.6696 - loss: 0.7336 - val_accuracy: 0.8889 - val_loss: 0.4550
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.6070 - loss: 0.8206 - val_accuracy: 0.8889 - val_loss: 0.4359
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.7517 - loss: 0.5727 - val_accuracy: 0.8889 - val_loss: 0.4133
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5826 - loss: 0.6872 - val_accuracy: 0.8889 - val_loss: 0.3912
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 776ms/step
Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 562ms/step - accuracy: 0.5865 - loss: 0.8641 - val_accuracy: 0.6111 - val_loss: 0.7139
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5982 - loss: 0.8721 - val_accuracy: 0.6111 - val_loss: 0.7151
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.7146 - loss: 0.6520 - val_accuracy: 0.6667 - val_loss: 0.6704
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.6187 - loss: 0.8371 - val_accuracy: 0.6667 - val_loss: 0.6265
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5904 - loss: 0.7067 - val_accuracy: 0.6667 - val_loss: 0.6119
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7194 - loss: 0.5600 - val_accuracy: 0.7222 - val_loss: 0.6050
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.6012 - loss: 0.7364 - val_accuracy: 0.7222 - val_loss: 0.5997
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.7263 - loss: 0

In [16]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.3f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.3f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.3f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.3f}".format(np.std(f1_score_per_run, axis=0))}')
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

Mean test accuracy is 0.857, mean test f1 score is 0.853, max test accuracy is 0.895, max test f1 score is 0.894, min test accuracy is 0.807, min test f1 score is 0.797, std of test accuracy is 0.037, std of test f1 score is 0.041
Time elapsed through all process: 369.855, sec
